In [ ]:
import csv
import requests
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from datetime import datetime
import matplotlib.lines as mlines
import matplotlib.offsetbox as offsetbox
import formatting as form

plt.close('all')

In [ ]:
CSV_URL = 'https://covidtracking.com/api/v1/states/daily.csv'

with requests.Session() as s:
    download = s.get(CSV_URL)
    download
    decoded_content = download.content.decode('utf-8')
    cr = pd.read_csv(StringIO(decoded_content))
    df = pd.DataFrame(cr)

In [ ]:
# Convert date integers to DateTime
date = pd.to_datetime(df['date'].astype(str), format='%Y%m%d')
#Applied in a new DataFrame
df = pd.DataFrame({'date':date, 'state':df['state'], 'positiveIncrease':df['positiveIncrease']})

In [ ]:
#List containing every state abbreviation (includes DC)
stateNames = ['WA', 'ID', 'MT', 'ND', 'MN','WI','MI','NY','VT', 'NH','ME',
             'OR','WY', 'SD', 'IA', 'IL', 'IN', 'OH', 'PA', 'MA', 'RI', 'CT', 'NJ',
             'DE', 'MD', 'DC', 'CA', 'NV', 'UT', 'CO', 'NE', 'MO', 'KS', 'OK', 'NM', 
              'AZ', 'TX', 'LA', 'AR', 'TN', 'MS', 'AL', 'KY', 'WV', 'VA', 'NC',
             'SC', 'GA', 'FL', 'AK', 'HI']

In [ ]:
# Create a new DataFrame with each state as a column and their positiveIncreases as rows, indexed by date.
# I'm sure there's a cleaner way to do this...
dfstates = []

for name in stateNames:
    sel = df[(df['state']==name)]
    sel.index=sel['date']
    dfstates.insert(len(dfstates), sel)
s = 'positiveIncrease'
dfgroup = pd.DataFrame()

for d in dfstates:
    dfgroup[d['state'][0]] = d[s]
dfgroup.head()

In [ ]:
#Compares weekly totals to the previous week for 7 weeks. Returns how many consecutive weeks saw a relative increase to the previous week.
def consecutiveWeeksIncreased(state):
    weeks = 0
    prev = 999999
    for week in range(7):
        thisWeekSum = sumRange(state, weeks * 7, weeks * 7 + 8)
        if thisWeekSum <= prev:
            prev = thisWeekSum
            weeks += 1
        else:
            return weeks
    return weeks

In [ ]:
# Returns the sum of contents within a list's range
def sumRange(L,a,b):                                                                                                                                                                                                
    sum = 0                                                                                                                                                                                                         
    for i in range(a,b+1,1):                                                                                                                                                                                        
        sum += L[i]                                                                                                                                                                                                  
    return sum      

In [127]:
#Create a subplot for the state by state abbreviation
def do_plot(ax, name):
    
    #Get color pair based on how many weeks the state has seen an increase in total cases
    weeksClimbing = consecutiveWeeksIncreased(dfgroup[name])
    colors = form.getContrastingColors(weeksClimbing)
    faceColor = colors[0]
    infoColor = colors[1]
    
    #Set the face color
    ax.set_facecolor(faceColor)
    
    #plot the data for this state
    ax.plot(dfgroup[name], color=(infoColor), linewidth=1)
    
    #remove the labels, grids & ticks for a cleaner look
    ax.set_xlabel('')
    ax.grid(False)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    
    #Display the state abbreviation title
    ax.set_title(name,horizontalalignment='left', color=(infoColor), fontweight="bold", fontsize=14, x=0.025, y=.775)
    
    #Display a subtle border
    form.formatBorder(ax)
    
    #Display a '100 new cases' threshold for scale
    ax.axhline(y=100, color='r', linestyle='-', linewidth=.2)

    box = dict(boxstyle='round,pad=0.1', fc=faceColor, ec=faceColor, alpha=0.7)

    #Latest increase    
    lastChangeInt = form.getLastIncreaseChange(name, dfgroup)
    #Latest increase vs previous day
    changeNote = '(' + form.getIncreaseSign(lastChangeInt) + str(lastChangeInt) + ')'
    ax.annotate(s= changeNote, xy=(.025, 0.581), xycoords='axes fraction', fontsize=11, color=form.getIncreaseColor(lastChangeInt), bbox = box)
    
    lastChange = f'{lastChangeInt:,}'

    ax.annotate(s= '+' + f'{int(dfgroup[name][0]):,}', xy=(.025, 0.7), xycoords='axes fraction', fontsize=11, color=infoColor, bbox = box)
    

    #Subtle label background (in progress)
    #box = offsetbox.AnchoredOffsetbox()

In [128]:
# Add a state to the grid by abbreviation
def add_state(figure, stateName, grid):
    state = figure.add_subplot(grid)
    do_plot(state, stateName)
    return state

In [129]:
#'Last updated on' string
now = datetime.now()
dt_string = now.strftime("%m/%d/%Y %H:%M:%S")
dt_string = "Last updated: " + dt_string

In [130]:
#Subtitle - date range
lastDate = date[0]
lastDate = lastDate = lastDate.month_name() + ' ' + str(lastDate.day)
shape = date.shape
firstDate = date[shape[0]-1]
firstDate = firstDate.month_name() + ' ' + str(firstDate.day)
dateRangeString = 'From ' + firstDate + ' to ' + lastDate

In [131]:
#Adding groups of states in vertical columns using a list of abbreviation strings
def add_column(fig, names, column, grid):
    row = 0
    for name in names:
        if(name == ""):
            row+=1
            continue
        else:
            add_state(fig, name, grid[row, column])
            row+=1

In [132]:
# Plot setup
plt.style.use('fivethirtyeight')

g = gridspec.GridSpec(11, 11)

fig = plt.figure(figsize=(30,25), facecolor=form.light)

#Title
fig.suptitle('US Daily COVID-19 Case Increases', y=0.85, fontsize=24, color=form.titleColor)
fig.text(.5, .822, s=dateRangeString, color = form.titleColor, horizontalAlignment = 'center')


x = .1
y =.75

#Create legend
redline = mlines.Line2D([], [], color=form.redBar, marker='', markersize=15, label='100 new cases per day')
twoWeeksIncrease = mlines.Line2D([],[], color = form.medLight, marker='s',markersize=30, label='Rising for 2 weeks')
threeWeeksIncrease = mlines.Line2D([],[], color = form.medium, marker='s',markersize=30, label='Rising for 3 weeks')
fourWeeksIncrease = mlines.Line2D([],[], color = form.medDark, marker='s',markersize=30, label='Rising for 4+ weeks')

legend = fig.legend(handles=[redline, twoWeeksIncrease, threeWeeksIncrease, fourWeeksIncrease], loc='upper left',borderaxespad=10, framealpha=0, labelspacing=2)

for t in legend.get_texts():
    t.set_color(form.labelColor)
    
# Create state subplots
add_column(fig, ['', '', 'WA', 'OR', 'CA',  '', '', 'HI'], 0, g)
add_column(fig, ['', '', 'ID', 'NV', 'UT', 'AZ', '', 'AK'], 1, g)
add_column(fig, ['','', 'MT', 'WY', 'CO', 'NM'], 2, g)
add_column(fig, ['','','ND','SD', 'NE', 'KS', 'OK', 'TX'], 3, g)
add_column(fig, ['', '', 'MN', 'IA', 'MO', 'AR', 'LA'], 4, g)
add_column(fig, ['', 'WI', 'IL', 'IN', 'KY', 'TN', 'MS'], 5, g)
add_column(fig, ['', '','MI', 'OH', 'WV', 'NC', 'AL'], 6, g)
add_column(fig, ['', '', '', 'PA', 'VA', 'SC', 'GA'], 7, g)
add_column(fig, ['', '', 'NY', 'NJ', 'MD', '', '', 'FL'], 8, g)
add_column(fig, ['', 'VT', 'MA', 'CT', 'DE'], 9, g)
add_column(fig, ['ME', 'NH', 'RI', 'DC'], 10, g)

#create guide info

ax = fig.axes[24]

form.createLabelGuide(ax, 'State', (0, .85), 'axes fraction', .9)
form.createLabelGuide(ax, 'New cases reported on ' + lastDate, (0.01, .73), 'axes fraction', .7)
form.createLabelGuide(ax, 'Relative to prev. day', (.01, .63), 'axes fraction', .5)
form.createLabelGuide(ax, '100 new cases', (date[date.shape[0]-100], 100), 'data', .3)

#'Last updated on'
fig.text(x=.05, y=.25, s=dt_string, color=form.labelColor)
#Data source label
fig.text(x=.98, y=.25, s='Data collected via covidtracking.com', color = form.labelColor, horizontalAlignment='right')

UnboundLocalError: local variable 'box' referenced before assignment